In [ ]:

!wget https://www.data.gouv.fr/fr/datasets/r/eceb9fb4-3ebc-4da3-828d-f5939712600a -O ../datagouv/department_latest.csv 2> /dev/null


Cette page montre l'évolution des données hospitalières par rapport à l'estimation faite dans [Estimation de la vigueur de l'épidémie à la fin du confinement](./).

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import math
from datetime import datetime, date, timedelta
from sklearn.linear_model import LinearRegression
import scipy

%matplotlib inline

# Afficher les dates au format jj/mm/yy et les mettre en index
def convertDate(isodate):
    l = isodate.split('-')
    return l[2]+"/"+l[1]+"/"+l[0][2:]



def computeAggregates(df):
    
    #extraire les données toutes classe d'age
    all_age = df[df["sursaud_cl_age_corona"] == "0"]

    # Pivot par jour - somme (sur tous les départements)
    data_france = all_age.groupby(["date_de_passage"]).agg('sum')


    data_france["date"] = data_france.index
    data_france["date"] = data_france["date"].apply(convertDate)
    data_france = data_france.set_index(["date"])
    return data_france


def addRollingHospit(df):
    # Remplacer les valeurs nulles par na (pour ne pas perturber la vue en echelle logarithmique)
    df.loc[df["nbre_hospit_corona"] == 0,"nbre_hospit_corona"] = np.nan
    
    # Calculer une moyenne glissante géométrique, centrée, avec une fenêtre de 7 pour lisser
    # la courbe des hospitalisations quotidiennes
    df["rolling_hospit_corona"] = df["nbre_hospit_corona"].rolling(7,center=True).aggregate(lambda x: x.prod()**(1./7))


# Charger les données utilisées pour la prédiction
df = pd.read_csv("../datagouv/sursaud-covid19-quotidien-2020-04-17-19h00-departement.csv")
reg_date = "17/04/2020"
data_france = computeAggregates(df)


# Ajouter la moyenne lissée pour calculer le maximum
addRollingHospit(data_france)

# Détermination du pic des hospitalisations sur la courbe lissée
peak = data_france["rolling_hospit_corona"].idxmax()
peak_loc = data_france.index.get_loc(peak)

# Calculer l'index de régression pour les données du 17 avril

# Sélectionner les données à partir de 2 avril (5 jours après le maximum)
for_regression = data_france.iloc[peak_loc+5:]
# Eliminer les jours sans données
for_regression = for_regression[for_regression["nbre_hospit_corona"] >= 0]
# Enlever aussi les deux derniers jours - car tous les chiffres ne sont pas toujours remontés
for_regression = for_regression.loc[for_regression.index[:-2]]
    
# AJouter les données les plus récentes    
df = pd.read_csv("../datagouv/department_latest.csv")
latest_data = computeAggregates(df)
# Data date is the latest  entry in the table
data_date = convertDate(df["date_de_passage"].iloc[-1])
    
# Ajouter les nouvelles données, en replacant les deux derniers jours
new_data = latest_data.iloc[latest_data.index.get_loc("15/04/20"):]

# drop last 2 days
data_france.drop("15/04/20", inplace=True)
data_france.drop("16/04/20", inplace=True)
# add the new data to the data used for the regression
data_france = data_france.append(new_data, sort=False)


# Agrandissement du dataframe jusqu'au 11 mai si nécessaire
d = data_france.index[-1]
a = d.split("/")
dd = int(a[0])
mm = int(a[1])
yy = 2000 + int(a[2])
first = date(yy,mm,dd)+ timedelta(days=1)
last = date(2020,5,11)

current = first
indexExtension  = []
while current  <= last:
    ds = str(current.day)
    if len(ds) == 1:
        ds = '0'+ds
    ms = str(current.month)
    if len(ms) == 1:
        ms = '0'+ms
    ys = str(current.year)[2:]
    di = ds + '/' + ms + '/' + ys
    indexExtension.append(di)
    current += timedelta(days = 1)

data_france = data_france.reindex(index = data_france.index.append(pd.Index(indexExtension)))

# Ajouter une colonne de numéro de jour
data_france["jour"] = np.arange(len(data_france))


# Faire une régression sur les du 17 avril


# Faire une régression linéaire sur les données
reg = LinearRegression()
for_regression["jour"] = data_france["jour"]
X_train = for_regression.drop(columns = [c for c in for_regression.columns if c != "jour"])
Y_train = np.log(for_regression["nbre_hospit_corona"])
reg.fit(X_train,Y_train)

# Ajouter la prédiction dans les données
data_france["pred_hosp"]=np.nan
# Plage de prédiction: dans la phase descendante - jusqu'au 11 mai max
predIndex = data_france[(data_france["jour"] >= X_train.iloc[0]["jour"])  & (data_france["jour"] <= data_france.loc["11/05/20"]["jour"])].index
X = data_france.loc[predIndex].drop(columns = [c for c in data_france.columns if c != "jour"])
data_france.loc[predIndex,"pred_hosp"]=np.exp(reg.predict(X))

# Calcul de l'intervalle de confiance de la prédiction
# Voir http://pageperso.lif.univ-mrs.fr/~alexis.nasr/Ens/IAAAM2/SlidesModStat_C1_print.pdf
def estimateSigma(reg, X, Y):
    Y_pred = reg.predict(X)
    err = (Y - Y_pred)**2
    return math.sqrt(err.sum() / (len(err) - 2))

sigma = estimateSigma(reg,X_train,Y_train)
X_train_mean = X_train["jour"].mean()
# Ajout de l'intervalle de confiance en log (alpha = 10% -- 1 - alpha/2 = 0.95)
data_france["conf_log_raw"] = np.nan
# Plage pour l'intervalle de confiance: depuis les données utilisées pour la régerssion linéaire
data_france.loc[predIndex,"conf_log_raw"] = np.sqrt(1 + 1./len(X_train) + \
                                           (data_france["jour"]-X_train_mean)**2 / ((X_train["jour"]-X_train_mean)**2).sum()) * \
                                           sigma*scipy.stats.t.ppf(0.95,len(X_train)-2)
data_france["pred_max"] = data_france["pred_hosp"]*np.exp(data_france["conf_log_raw"])
data_france["pred_min"] = data_france["pred_hosp"]/np.exp(data_france["conf_log_raw"])



# Enlever les prédictions dans la zone de regression
data_france.loc[for_regression.index,"pred_hosp"] = np.nan

# Mettre à jour la moyenne glissante
addRollingHospit(data_france)



confinement_start = "17/03/20"
start_loc = data_france.index.get_loc(confinement_start)

def plotData(useLog = True):
    # Afficher les courbes
    fig = plt.figure(figsize=(15,8))
    ax = plt.axes()

    beforeIndex = data_france.iloc[:data_france.index.get_loc("16/04/20")].index
    afterIndex = data_france.iloc[data_france.index.get_loc("16/04/20") - 1:].index
    ax.plot(data_france.loc[beforeIndex, "nbre_hospit_corona"], label="Nouvelles hospitalisations quotidienne")
    ax.plot(data_france.loc[beforeIndex,"rolling_hospit_corona"], label="Nouvelles hospitalisations quotidienne lissées")
    ax.plot(data_france["pred_hosp"], "--", label="Prédiction hospitalisations quotidienne avec les données au 17 avril")
    ax.plot(data_france.loc[afterIndex, "nbre_hospit_corona"], label="Nouvelles hospitalisations quotidienne après le 17 avril")
    ax.plot(data_france.loc[afterIndex,"rolling_hospit_corona"], label="Nouvelles hospitalisations quotidienne lissées après le 17 avril")
    ax.fill_between(data_france.index, data_france["pred_max"], data_france["pred_min"],color="green",alpha=0.3)
    #ax.plot(data_france["nv_cas"], "--", label="Estimation nouveaux cas nécessitant hospitalisation après 11 jours") 
    
    ax.xaxis.set_major_locator(plt.MaxNLocator(10))
    if useLog:
        ax.set_title("Hospitalisations COVID-19 quotidiennes en France en échelle logarithmique")
    else:
        ax.set_title("Hospitalisations COVID-19 quotidiennes en France en échelle linéaire")
        
    ax.text(start_loc,data_france["rolling_hospit_corona"].loc[confinement_start],"Début du confinement\n%s"%confinement_start,horizontalalignment="center")
    ax.text(peak_loc,data_france["rolling_hospit_corona"].loc[peak],"Maximum \n%s\naprès %i jours"%(peak,peak_loc-start_loc),horizontalalignment="center")

    ax.text(data_france.index[-1],data_france["pred_hosp"].iloc[-1],"%i"%int(data_france["pred_hosp"].iloc[-1]),horizontalalignment="center")
    
    #ax.text(data_france.index[-1],data_france["nv_cas"].iloc[-1],"%i"%int(data_france["nv_cas"].iloc[-1]),horizontalalignment="center")
    
    ax.legend()
    if useLog:
        plt.yscale("log")

    fig.text(.5, .05, "D'après les données des urgences hospitalières du %s"%data_date, ha='center')

plotData(useLog=True)
plotData(useLog=False)

